In [18]:
import torch
import json
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from datasets import load_dataset
from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
from transformers import AutoModel, AutoTokenizer
from huggingface_hub import HfApi
from huggingface_hub import upload_file, delete_file,upload_folder


In [2]:
from huggingface_hub import notebook_login
#huggingface upload
notebook_login()

Login successful
Your token has been saved to C:\Users\DESKTOP/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [3]:
def readKLUETrainData(path):
    trainDataset=[]
    with open(path,'rb')as file:
        STSdata=json.load(file)
    
    for item in STSdata:
        score=float(item['labels']['label'])/5.0

        inputEx=InputExample(
        texts=[item["sentence1"],item["sentence2"]],
        label=score
        )
        trainDataset.append(inputEx)
    
    return trainDataset

In [4]:
def readNLITrainData(path):
    trainDataset=pd.DataFrame({"sentence1":[],
                              "sentence2":[],
                              "score":[]})
    df = pd.read_csv(dataPath, on_bad_lines='skip')
    
    trainDataset["sentence1"]=df["sentence1"]
    trainDataset["sentence2"]=df["sentence2"]
    trainDataset["score"]=df["score"]
    
    return trainDataset

In [8]:
BATCH_SIZE=32
EPOCH=4
outputPath=("TAACO_STS_epoch-"+str(EPOCH)).replace("/","-")
outputPath='./outputs/'+outputPath

In [9]:
model = SentenceTransformer("KDHyun08/TAACO_STS")

C:\Users\DESKTOP\anaconda3\envs\a2\lib\site-packages\huggingface_hub\file_download.py:563: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  FutureWarning,


In [10]:
dataPath="./dataset/train.csv"

trainDataset=readNLITrainData(dataPath)
train_input=[]
try:
    for item in trainDataset.loc():
        inputEx=InputExample(
                texts=[item["sentence1"],item["sentence2"]],
                label=float(item["score"]/5.0)
                )
        train_input.append(inputEx)
except:
    print(len(train_input))


4538


In [11]:
trainDataLoader=DataLoader(
train_input,
shuffle=True,
batch_size=BATCH_SIZE
)
trainLoss=losses.CosineSimilarityLoss(model=model)

In [12]:
evaluator=EmbeddingSimilarityEvaluator.from_input_examples(
train_input,
name="sts-vali"
)

In [14]:
model.fit(
train_objectives=[(trainDataLoader, trainLoss)],
    evaluator=evaluator,
    epochs=EPOCH,
    evaluation_steps=1000,
    output_path=outputPath
 )

C:\Users\DESKTOP\anaconda3\envs\a2\lib\site-packages\transformers\optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/142 [00:00<?, ?it/s]

Iteration:   0%|          | 0/142 [00:00<?, ?it/s]

Iteration:   0%|          | 0/142 [00:00<?, ?it/s]

Iteration:   0%|          | 0/142 [00:00<?, ?it/s]

In [16]:
#tokenizer.save_pretrained("./model")
model.save("./model")

In [19]:
upload_folder(
    folder_path ="model",
    path_in_repo="",
    repo_id="KDHyun08/TAACO_STS",
)

'https://huggingface.co/KDHyun08/TAACO_STS/tree/main/'